# Weather data augmentation - Chicago 311 Response Time Prediction

## Setup: Install Dependencies

In [17]:
# Run this cell only if libraries are not installed
# !pip install pandas numpy scikit-learn requests holidays geopy matplotlib seaborn

## Imports

In [2]:
import pandas as pd
import numpy as np
import requests
import holidays
from geopy.distance import geodesic
from sklearn.model_selection import KFold
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## Load and Prepare Base Dataset

In [3]:
# Load raw data from git-lfs
data_path = Path('../data/raw/311_Service_Requests_Since_2020.csv')

print(f"Loading data from: {data_path}")
print(f"File exists: {data_path.exists()}")

# Read CSV
df = pd.read_csv(data_path)

print(f"\n✓ Loaded {len(df):,} rows")
print(f"✓ Columns: {df.shape[1]}")
print(f"\nColumn names:")
print(df.columns.tolist())

Loading data from: ../data/raw/311_Service_Requests_Since_2020.csv
File exists: True

✓ Loaded 2,229,793 rows
✓ Columns: 28

Column names:
['SR_NUMBER', 'SR_TYPE', 'SR_SHORT_CODE', 'ORIGIN', 'CREATED_DATE', 'CLOSED_DATE', 'ZIP_CODE', 'COMMUNITY_AREA', 'WARD', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK', 'CREATED_MONTH', 'X_COORDINATE', 'Y_COORDINATE', 'LATITUDE', 'LONGITUDE', 'CREATED_DEPARTMENT', 'OWNER_DEPARTMENT', 'ELECTRICAL_DISTRICT', 'ELECTRICITY_GRID', 'POLICE_SECTOR', 'POLICE_DISTRICT', 'POLICE_BEAT', 'PRECINCT', 'STREET_NUMBER', 'STREET_DIRECTION', 'STREET_NAME', 'STREET_TYPE']


In [4]:
# Display first few rows
df.head()

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,ORIGIN,CREATED_DATE,CLOSED_DATE,ZIP_CODE,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,X_COORDINATE,Y_COORDINATE,LATITUDE,LONGITUDE,CREATED_DEPARTMENT,OWNER_DEPARTMENT,ELECTRICAL_DISTRICT,ELECTRICITY_GRID,POLICE_SECTOR,POLICE_DISTRICT,POLICE_BEAT,PRECINCT,STREET_NUMBER,STREET_DIRECTION,STREET_NAME,STREET_TYPE
0,SR20-03236940,Graffiti Removal Request,GRAF,Mobile Device,01/01/2020 12:17:00 AM,01/02/2020 09:06:28 AM,60616.0,34.0,25.0,0,4,1,1.174071e+06,1.889129e+06,41.851184,-87.636608,NaN,Streets and Sanitation,NaN,P027,1.0,9.0,914.0,18.0,2263,S,STEWART,AVE
1,SR20-03236942,Rodent Baiting/Rat Complaint,SGA,Mobile Device,01/01/2020 12:19:30 AM,01/02/2020 10:12:43 AM,60632.0,57.0,14.0,0,4,1,1.148548e+06,1.871197e+06,41.802506,-87.730747,NaN,Streets and Sanitation,NaN,F033,1.0,8.0,815.0,8.0,4955,S,KILDARE,AVE
2,SR20-03236943,Garbage Cart Maintenance,SIE,Internet,01/01/2020 12:19:48 AM,02/20/2020 09:26:26 AM,60645.0,2.0,50.0,0,4,1,1.159299e+06,1.949262e+06,42.016509,-87.689169,NaN,Streets and Sanitation,NaN,K004,1.0,24.0,2411.0,8.0,7428,N,CLAREMONT,AVE
3,SR20-03236974,Abandoned Vehicle Complaint,SKA,Internet,01/01/2020 12:43:32 AM,02/07/2020 02:18:57 PM,60656.0,10.0,41.0,0,4,1,1.128611e+06,1.935354e+06,41.978924,-87.802411,NaN,Streets and Sanitation,NaN,Y109,1.0,16.0,1613.0,45.0,5360,N,NORDICA,AVE
4,SR20-03236988,Traffic Signal Out Complaint,SFB,Mobile Device,01/01/2020 12:55:40 AM,01/03/2020 09:51:41 AM,60622.0,22.0,32.0,0,4,1,1.164375e+06,1.913327e+06,41.917796,-87.671512,NaN,CDOT - Department of Transportation,NaN,L018,3.0,14.0,1433.0,33.0,1959,N,HERMITAGE,AVE


In [5]:
# Check data types and missing values
print("Data Info:")
df.info()

print("\nMissing values:")
print(df.isnull().sum())

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229793 entries, 0 to 2229792
Data columns (total 28 columns):
 #   Column               Dtype  
---  ------               -----  
 0   SR_NUMBER            object 
 1   SR_TYPE              object 
 2   SR_SHORT_CODE        object 
 3   ORIGIN               object 
 4   CREATED_DATE         object 
 5   CLOSED_DATE          object 
 6   ZIP_CODE             float64
 7   COMMUNITY_AREA       float64
 8   WARD                 float64
 9   CREATED_HOUR         int64  
 10  CREATED_DAY_OF_WEEK  int64  
 11  CREATED_MONTH        int64  
 12  X_COORDINATE         float64
 13  Y_COORDINATE         float64
 14  LATITUDE             float64
 15  LONGITUDE            float64
 16  CREATED_DEPARTMENT   object 
 17  OWNER_DEPARTMENT     object 
 18  ELECTRICAL_DISTRICT  float64
 19  ELECTRICITY_GRID     object 
 20  POLICE_SECTOR        float64
 21  POLICE_DISTRICT      float64
 22  POLICE_BEAT          float64
 23  PRECINCT             

In [6]:
# Convert date columns to datetime
print("Converting date columns...")

# Check the actual format of the dates
print(f"Sample CREATED_DATE: {df['CREATED_DATE'].iloc[0]}")
print(f"Sample CLOSED_DATE: {df['CLOSED_DATE'].iloc[0] if pd.notna(df['CLOSED_DATE'].iloc[0]) else 'NaN'}")

# Convert to datetime (adjust format if needed based on sample)
df['CREATED_DATE'] = pd.to_datetime(df['CREATED_DATE'], errors='coerce')
df['CLOSED_DATE'] = pd.to_datetime(df['CLOSED_DATE'], errors='coerce')

print(f"\n✓ Converted dates")
print(f"CREATED_DATE range: {df['CREATED_DATE'].min()} to {df['CREATED_DATE'].max()}")
print(f"CLOSED_DATE range: {df['CLOSED_DATE'].min()} to {df['CLOSED_DATE'].max()}")

Converting date columns...
Sample CREATED_DATE: 01/01/2020 12:17:00 AM
Sample CLOSED_DATE: 01/02/2020 09:06:28 AM


/var/folders/jv/2mp5gk6d2c54cp2653crmxw80000gn/T/ipykernel_89180/16862326.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CREATED_DATE'] = pd.to_datetime(df['CREATED_DATE'], errors='coerce')



✓ Converted dates
CREATED_DATE range: 2020-01-01 00:17:00 to 2025-09-30 20:13:05
CLOSED_DATE range: 2020-01-01 09:28:15 to 2025-09-30 20:12:47


In [7]:
# Extract base temporal features
df['CREATED_HOUR'] = df['CREATED_DATE'].dt.hour
df['CREATED_DAY_OF_WEEK'] = df['CREATED_DATE'].dt.dayofweek  # 0=Monday, 6=Sunday
df['CREATED_MONTH'] = df['CREATED_DATE'].dt.month
df['CREATED_DATE_ONLY'] = df['CREATED_DATE'].dt.date

print("✓ Extracted base temporal features:")
print(f"  - CREATED_HOUR (0-23)")
print(f"  - CREATED_DAY_OF_WEEK (0-6, Monday-Sunday)")
print(f"  - CREATED_MONTH (1-12)")
print(f"  - CREATED_DATE_ONLY (for weather merge)")

print(f"\nSample temporal features:")
df[['CREATED_DATE', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK', 'CREATED_MONTH']].head()

✓ Extracted base temporal features:
  - CREATED_HOUR (0-23)
  - CREATED_DAY_OF_WEEK (0-6, Monday-Sunday)
  - CREATED_MONTH (1-12)
  - CREATED_DATE_ONLY (for weather merge)

Sample temporal features:


,CREATED_DATE,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
0,2020-01-01 00:17:00,0,2,1
1,2020-01-01 00:19:30,0,2,1
2,2020-01-01 00:19:48,0,2,1
3,2020-01-01 00:43:32,0,2,1
4,2020-01-01 00:55:40,0,2,1


## Weather Data Integration

In [8]:
def fetch_chicago_weather(start_date, end_date, cache_path):
    """
    Fetch historical weather data for Chicago from Open-Meteo API.

    Args:
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
        cache_path: Path to cache the weather data

    Returns:
        pd.DataFrame: Daily weather data
    """
    # Check if cache exists
    if Path(cache_path).exists():
        print(f"Loading cached weather data from {cache_path}...")
        weather_df = pd.read_csv(cache_path, parse_dates=['date'])
        print(f"✓ Loaded {len(weather_df)} days from cache")
        return weather_df

    print(f"Fetching weather data from Open-Meteo API ({start_date} to {end_date})...")

    url = "https://archive-api.open-meteo.com/v1/archive"

    # Only fetch features that matter for prediction:
    # - Temperature (work speed, equipment function)
    # - Precipitation (delays outdoor work)
    # - Snow (road access, delays)
    params = {
        "latitude": 41.8781,  # Chicago Loop
        "longitude": -87.6298,
        "start_date": start_date,
        "end_date": end_date,
        "daily": [
            "temperature_2m_mean",      # Daily mean temp
            "precipitation_sum",         # Total precipitation
            "snowfall_sum",             # Total snowfall
        ],
        "temperature_unit": "fahrenheit",
        "precipitation_unit": "inch",
        "timezone": "America/Chicago"
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Weather API error: {response.status_code}\n{response.text}")

    data = response.json()

    # Convert to DataFrame - only keep essential features
    weather_df = pd.DataFrame({
        'date': pd.to_datetime(data['daily']['time']),
        'temp_mean': data['daily']['temperature_2m_mean'],
        'precipitation': data['daily']['precipitation_sum'],
        'snowfall': data['daily']['snowfall_sum'],
    })

    # Cache the results
    weather_df.to_csv(cache_path, index=False)
    print(f"✓ Fetched {len(weather_df)} days of weather data")
    print(f"✓ Cached to {cache_path}")

    return weather_df

# Determine date range from our data
min_date = df['CREATED_DATE'].min().date()
max_date = df['CREATED_DATE'].max().date()

print(f"Data date range: {min_date} to {max_date}")
print(f"Total days needed: {(max_date - min_date).days + 1}")

# Fetch weather for full date range
cache_path = Path('../data/raw/chicago_weather_full.csv')
weather_df = fetch_chicago_weather(
    start_date=str(min_date),
    end_date=str(max_date),
    cache_path=cache_path
)

print("\nWeather data preview:")
print(weather_df.head(10))
print("\nWeather statistics:")
print(weather_df.describe())

Data date range: 2020-01-01 to 2025-09-30
Total days needed: 2100
Fetching weather data from Open-Meteo API (2020-01-01 to 2025-09-30)...
✓ Fetched 2100 days of weather data
✓ Cached to ../data/raw/chicago_weather_full.csv

Weather data preview:
        date  temp_mean  precipitation  snowfall
0 2020-01-01       31.1          0.000     0.000
1 2020-01-02       40.8          0.000     0.000
2 2020-01-03       34.6          0.000     0.000
3 2020-01-04       30.6          0.079     0.551
4 2020-01-05       31.7          0.004     0.028
5 2020-01-06       31.7          0.000     0.000
6 2020-01-07       32.4          0.008     0.055
7 2020-01-08       20.8          0.000     0.000
8 2020-01-09       38.6          0.012     0.000
9 2020-01-10       41.0          0.748     0.000

Weather statistics:
                                date    temp_mean  precipitation     snowfall
count                           2100  2100.000000    2100.000000  2100.000000
mean   2022-11-15 12:00:00.000000256  

In [9]:
# Prepare weather data for merge (date only, no time)
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date

# Merge weather data to main dataset
print(f"Merging weather data...")
print(f"  Main dataset rows before merge: {len(df):,}")
print(f"  Weather data dates: {len(weather_df):,}")

df = df.merge(weather_df, left_on='CREATED_DATE_ONLY', right_on='date', how='left')

print(f"  Main dataset rows after merge: {len(df):,}")

# Check for missing weather data
missing_weather = df['temp_mean'].isna().sum()
print(f"  Rows with missing weather: {missing_weather:,} ({missing_weather/len(df)*100:.2f}%)")

if missing_weather > 0:
    # Show which dates are missing
    missing_dates = df[df['temp_mean'].isna()]['CREATED_DATE_ONLY'].unique()
    print(f"  Missing dates sample: {sorted(missing_dates)[:5]}")

print("\n✓ Weather data merged successfully")

Merging weather data...
  Main dataset rows before merge: 2,229,793
  Weather data dates: 2,100
  Main dataset rows after merge: 2,229,793
  Rows with missing weather: 0 (0.00%)

✓ Weather data merged successfully


In [10]:
# Create simplified derived weather features (only predictive ones)
print("Creating derived weather features...\n")

# 1. Extreme cold (below 20°F mean) - impacts work speed and equipment
df['extreme_cold'] = (df['temp_mean'] < 20).fillna(0).astype(int)
extreme_cold_count = (df['temp_mean'] < 20).sum()
print(f"✓ extreme_cold: {extreme_cold_count:,} requests on extreme cold days")

# 2. Heavy precipitation (>0.5 inches) - delays outdoor work
df['heavy_precipitation'] = (df['precipitation'] > 0.5).fillna(0).astype(int)
heavy_precip_count = (df['precipitation'] > 0.5).sum()
print(f"✓ heavy_precipitation: {heavy_precip_count:,} requests on heavy rain days")

# 3. Snow day (any measurable snowfall) - affects road access and work
df['snow_day'] = (df['snowfall'] > 0).fillna(0).astype(int)
snow_day_count = (df['snowfall'] > 0).sum()
print(f"✓ snow_day: {snow_day_count:,} requests on snow days")

# 4. Temperature deviation from monthly normal - unusual weather disrupts operations
# Chicago average temps by month (historical normals)
monthly_normal_temps = {
    1: 27, 2: 30, 3: 41, 4: 52, 5: 63, 6: 72,
    7: 77, 8: 75, 9: 68, 10: 56, 11: 43, 12: 31
}
df['temp_normal'] = df['CREATED_MONTH'].map(monthly_normal_temps)
df['temp_deviation'] = (df['temp_mean'] - df['temp_normal']).fillna(0)

print(f"✓ temp_deviation calculated")
print(f"  Range: {df['temp_deviation'].min():.1f}°F to {df['temp_deviation'].max():.1f}°F")

print("\n✓ Created 4 derived weather features (simplified)")

Creating derived weather features...

✓ extreme_cold: 69,533 requests on extreme cold days
✓ heavy_precipitation: 158,404 requests on heavy rain days
✓ snow_day: 200,641 requests on snow days
✓ temp_deviation calculated
  Range: -35.9°F to 27.9°F

✓ Created 4 derived weather features (simplified)


In [11]:
# Summary of all weather features
weather_features = [
    # Base weather metrics (3)
    'temp_mean',
    'precipitation',
    'snowfall',
    # Derived features (4)
    'extreme_cold',
    'heavy_precipitation',
    'snow_day',
    'temp_deviation'
]

print("\n" + "="*60)
print("WEATHER DATA AUGMENTATION COMPLETE (FIXED)")
print("="*60)
print(f"\nTotal weather features added: {len(weather_features)}")
print("\nFeature list (simplified for prediction):")
for i, feat in enumerate(weather_features, 1):
    print(f"  {i}. {feat}")

print(f"\nCurrent dataset shape: {df.shape}")
print(f"Total rows: {len(df):,}")
print(f"\nMissing weather data: {df['temp_mean'].isna().sum():,} ({df['temp_mean'].isna().sum()/len(df)*100:.2f}%)")


WEATHER DATA AUGMENTATION COMPLETE (FIXED)

Total weather features added: 7

Feature list (simplified for prediction):
  1. temp_mean
  2. precipitation
  3. snowfall
  4. extreme_cold
  5. heavy_precipitation
  6. snow_day
  7. temp_deviation

Current dataset shape: (2229793, 38)
Total rows: 2,229,793

Missing weather data: 0 (0.00%)


## Verify Weather Merge Quality

Let's check the quality of the merge and see if weather data is properly aligned.

In [12]:
# Check merge quality
print("Weather Data Merge Quality Check:\n")

# Count records with weather data
has_weather = df['temp_mean'].notna().sum()
missing_weather = df['temp_mean'].isna().sum()

print(f"Records with weather data: {has_weather:,} ({has_weather/len(df)*100:.2f}%)")
print(f"Records missing weather:   {missing_weather:,} ({missing_weather/len(df)*100:.2f}%)")

# Check distribution across years
print("\nRecords by year:")
year_stats = df.groupby(df['CREATED_DATE'].dt.year).agg({
    'SR_NUMBER': 'count',
    'temp_mean': lambda x: x.notna().sum()
}).rename(columns={'SR_NUMBER': 'total', 'temp_mean': 'with_weather'})
year_stats['pct_with_weather'] = (year_stats['with_weather'] / year_stats['total'] * 100).round(2)
print(year_stats)

# Sample of data with weather
print("\nSample of merged data (with weather):")
print(df[df['temp_mean'].notna()][['CREATED_DATE', 'temp_mean', 'precipitation', 'snowfall', 'extreme_cold', 'snow_day']].head(10))

# If missing data, show which dates
if missing_weather > 0:
    print(f"\nDates with missing weather (sample):")
    missing_dates = sorted(df[df['temp_mean'].isna()]['CREATED_DATE_ONLY'].unique())
    print(f"  First 10: {missing_dates[:10]}")
    print(f"  Last 10: {missing_dates[-10:]}")

Weather Data Merge Quality Check:

Records with weather data: 2,229,793 (100.00%)
Records missing weather:   0 (0.00%)

Records by year:
               total  with_weather  pct_with_weather
CREATED_DATE                                        
2020          427258        427258             100.0
2021          390988        390988             100.0
2022          367845        367845             100.0
2023          359983        359983             100.0
2024          389203        389203             100.0
2025          294516        294516             100.0

Sample of merged data (with weather):
         CREATED_DATE  temp_mean  precipitation  snowfall  extreme_cold  \
0 2020-01-01 00:17:00       31.1            0.0       0.0             0   
1 2020-01-01 00:19:30       31.1            0.0       0.0             0   
2 2020-01-01 00:19:48       31.1            0.0       0.0             0   
3 2020-01-01 00:43:32       31.1            0.0       0.0             0   
4 2020-01-01 00:55:40    

In [13]:
# Weather feature statistics
print("Weather Feature Statistics (non-null values only):\n")

weather_cols = ['temp_mean', 'precipitation', 'snowfall', 'extreme_cold', 'heavy_precipitation', 'snow_day', 'temp_deviation']

for col in weather_cols:
    if col in df.columns:
        non_null = df[col].notna().sum()
        print(f"{col}:")
        if df[col].dtype in ['int64', 'float64']:
            print(f"  Count: {non_null:,}")
            print(f"  Mean: {df[col].mean():.2f}")
            print(f"  Min: {df[col].min():.2f}")
            print(f"  Max: {df[col].max():.2f}")
        print()

Weather Feature Statistics (non-null values only):

temp_mean:
  Count: 2,229,793
  Mean: 54.38
  Min: -7.00
  Max: 86.90

precipitation:
  Count: 2,229,793
  Mean: 0.12
  Min: 0.00
  Max: 5.42

snowfall:
  Count: 2,229,793
  Mean: 0.05
  Min: 0.00
  Max: 4.52

extreme_cold:
  Count: 2,229,793
  Mean: 0.03
  Min: 0.00
  Max: 1.00

heavy_precipitation:
  Count: 2,229,793
  Mean: 0.07
  Min: 0.00
  Max: 1.00

snow_day:
  Count: 2,229,793
  Mean: 0.09
  Min: 0.00
  Max: 1.00

temp_deviation:
  Count: 2,229,793
  Mean: -1.84
  Min: -35.90
  Max: 27.90



## Save Progress: Weather-Augmented Dataset

In [14]:
# Save intermediate result with weather features
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / '311_Service_Requests_Since_2020_with_weather.csv'
df.to_csv(output_path, index=False)

print(f"✓ Saved weather-augmented dataset to {output_path}")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {df.shape[1]}")
print(f"  Weather features: {len(weather_features)}")

✓ Saved weather-augmented dataset to ../data/processed/311_Service_Requests_Since_2020_with_weather.csv
  Rows: 2,229,793
  Columns: 38
  Weather features: 7


**Weather Features Added:**
- **Base metrics (3):** temp_mean, precipitation, snowfall
- **Derived flags (4):** extreme_cold, heavy_precipitation, snow_day, temp_deviation